In [2]:
# postServer-mqtt client
import paho.mqtt.client as mqtt
from bigchaindb_driver.crypto import generate_keypair
import json

# 提交事务
def transactionsPost(dictData):
#     dict_data = {
#     "operation":"TRANSFER",
#     "asset":{"id":"123456"},
#     "signers"="admin_public_key"
#     "inputs":[{"input":"input_msg"}],
#     "recipients": [[["public_key1"],2],[["public_key2",6]]],
#     "metadata": {"a":1},
#     "private_keys": ["p1","p2"]
#     }

    tuple_recipients = []
    for i in dict_data["recipients"]:
        new_data = (i,)
        tuple_recipients.append(new_data)
    # 准备事务
    #  operation='CREATE',
    if dict_data["operation"] == 'CREATE':
        prepared_transfer_tx = bdb.transactions.prepare(
         operation=dict_data["operation"],
         asset=dict_data["asset"],
         signers=dict_data["signers"],
         recipients=tuple_recipients,
         metadata=dict_data["metadata"]

    )
    #  operation='TRANSFER',
    if dict_data["operation"] == 'TRANSFER':
        prepared_transfer_tx = bdb.transactions.prepare(
             operation=dict_data["operation"],
             asset=dict_data["asset"],
             inputs=dict_data["inputs"],
             recipients=tuple_recipients,
             metadata=dict_data["metadata"]

        )
    # 提交事务
    fulfilled_transfer_tx = bdb.transactions.fulfill(prepared_transfer_tx, private_keys=dict_data["private_keys"])
    # 验证提交结果
    if fulfilled_transfer_tx == prepared_transfer_tx:
        return fulfilled_transfer_tx
    return {"errMsg":"事务提交失败"}
    

# 生成密钥对
def generateKeypair():
    # generate a keypair
    keypair = generate_keypair()
    return {"publicKey": str(keypair.public_key),"privateKey": str(keypair.private_key)}

def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))


def on_message(client, userdata, msg):
    print(msg.topic + " " + str(msg.payload))
    
    reqTopic = str(msg.topic)
#     print(reqTopic, type(reqTopic))
    reqPayload = json.loads(msg.payload)  # 解码请求参数
    reqId = reqPayload["clientId"]
    repTopic = reqTopic.replace("postServer", reqId)  # 生成响应topic
    
    if reqTopic == "postServer/secretKey":
        print(reqId+"生成密钥对")
        # 生成密钥对；
        print(json.dumps(generateKeypair()))
        client.publish(repTopic, payload=json.dumps(generateKeypair()), qos=0)
        return

    if reqTopic == "postServer/transaction":
        print("提交transaction")
        # chain 事务提交
        client.publish(repTopic, payload=json.dumps(transactionsPost(reqPayload)), qos=0)
        return
    
    print("undefine topic" + repTopic)
    return


client = mqtt.Client("postServer")
client.on_connect = on_connect
client.on_message = on_message
client.connect("192.168.113.6", 1883, 60)

client.subscribe('postServer/secretKey', qos=0)  # 请求密钥对
client.subscribe('postServer/transaction', qos=0)  # 提交transaction

# client.publish('block/emqtt', payload='Hello,EMQ!', qos=0)
client.loop_forever()


Connected with result code 0
postServer/secretKey b'{"clientId":"BLOCk.1.B"}'
BLOCk.1.B生成密钥对
{"publicKey": "BHAMi21vEqpx6P8kGYQpoaGmD6qjoFfZsLgNPV1XZQkr", "privateKey": "Eif1Ni492PbT6KSgn6QnEat34NDkNBW8ZvEFqNWQZmzB"}


KeyboardInterrupt: 

In [2]:
# test def generateKeypair()
from bigchaindb_driver.crypto import generate_keypair
import json

# 生成密钥对
def generateKeypair():
    # generate a keypair
    keypair = generate_keypair()
    return json.dumps({"publicKey": str(keypair.public_key),"privateKey": str(keypair.private_key)})

createKeypair()

'{"publicKey": "CiDPR8eWMWSBKBJWwB6aTFoShiTLiNC5hUPLHSNFjWNT", "privateKey": "CpoaYJKM8rr89tnGVd2NbykNAMfsuVz7iadmLvRbaKL4"}'